# Optimising the regularisation parameter
--- 
Running ppxf with regularisation is very time-intensive and makes it difficult to run simulations across a large parameter space. 

Up until now we've been using $\Delta\chi^2 < 1$ as our condition for terminating regularisation, and an upper limit of `regul` = 50,000. 
* **Can we find a better maximum value for `regul`?** How does the solution change between 50,000 and e.g. 100,000? If there is no change, does this hold for SFHs of all shapes?
* **Can we find a less strict convergence criterion?** How does a solution with $\Delta\chi^2 < 1$ differ from a solution with $\Delta\chi^2 \sim 10$, for instance? Is there a meaningful difference?  

In [3]:
%matplotlib widget

In [4]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))
display(HTML("<style>.output_result { max-width:75% !important; }</style>"))

In [60]:
import os
import numpy as np
from numpy.random import RandomState
from time import time 
from tqdm.notebook import tqdm
from itertools import product
import multiprocessing
import pandas as pd
import extinction

from astropy.io import fits

from ppxftests.run_ppxf import run_ppxf
from ppxftests.ssputils import load_ssp_templates, get_bin_edges_and_widths
from ppxftests.mockspec import create_mock_spectrum
from ppxftests.sfhutils import load_sfh, compute_mw_age, compute_lw_age, compute_cumulative_mass, compute_cumulative_light
from ppxftests.sfhutils import compute_mean_age, compute_mean_mass, compute_mean_sfr, compute_mean_1D_sfh
from ppxftests.ppxf_plot import plot_sfh_mass_weighted, plot_sfh_light_weighted, ppxf_plot
from ppxftests.plot_ppxf_summary import plot_ppxf_summary

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
plt.ion()
plt.close("all")

from IPython.core.debugger import Tracer

fig_path = "/priv/meggs3/u5708159/ppxftests/figs/"
data_path = "/priv/meggs3/u5708159/ppxftests/"


In [15]:
##################################################################
# Settings
##################################################################
isochrones = "Padova"
SNR = 100
z = 0
niters = 20
nthreads = 20


In [84]:
##################################################################
# Define the input SFH
##################################################################
plt.close("all")
gal = 15
sfh_mw_input, sfh_lw_input, sfr_avg_input, sigma_star_kms = load_sfh(gal, plotit=True)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [85]:
###############################################################################
# Create spectrum
###############################################################################
spec, spec_err, lambda_vals_A = create_mock_spectrum(
    sfh_mass_weighted=sfh_mw_input,
    agn_continuum=False,
    isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
    plotit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [86]:
###########################################################################
# Helper function for running ppxf in parallel
###########################################################################
def ppxf_helper(regul):
    # Run ppxf
    pp = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                  z=z, ngascomponents=1, mdegree=-1, reddening=None,
                  regularisation_method="fixed", regul_fixed=regul, 
                  fit_agn_cont=False,
                  isochrones="Padova",
                  fit_gas=False, tie_balmer=True,
                  plotit=False, savefigs=False, interactive_mode=False)
    return pp


## How does the solution change as a function of `regul`? What happens at very high values of `regul`?
---

In [19]:
# Input arguments
regul_vals = np.logspace(-1, 7, 9)
regul_vals[0] = 0

# Run in parallel
print(f"Running ppxf on {nthreads} threads...")
t = time()
nthreads = len(regul_vals)
niters = len(regul_vals)
with multiprocessing.Pool(nthreads) as pool:
    pp_list = list(tqdm(pool.imap(ppxf_helper, regul_vals), total=niters))
print(f"Total time in ppxf: {time() - t:.2f} s")


Running ppxf on 20 threads...


----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 6.80 s
Iteration 1: Scaling noise by 1.3822...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 6.95 s
Iteration 1: Scaling noise by 1.3822...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 7.60 s
Iteration 1: Scaling noise by 1.3822...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 7.83 s
Iteration 1: Scaling noise by 1.3822...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 7.69 s
Iteration 1: Scaling noise by 1.3822...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 8.20 s
Iteration 1: Scaling noise by 1.3822...
----------------------------------------------------
-----------------------------------------

In [21]:
########################################################################
# Plot the SFH from each run
########################################################################
fig = plt.figure(figsize=(10, 3.5))
ax = fig.add_axes([0.1, 0.25, 0.8, 0.65])

cmap_regul = matplotlib.cm.get_cmap("Spectral", len(regul_vals))
for ii, pp in enumerate(pp_list):
    ax.step(pp.ages, np.log10(pp.sfh_mw_1D), where="mid", color=cmap_regul(ii), label=f"regul = {pp.regul}")   

# Decorations
ax.set_ylabel(r"Stellar mass ($\rm M_\odot$)")
ax.set_xlabel("Age (yr)")
ax.set_xscale("log")
ax.grid()
ax.legend(fontsize="small")
ax.autoscale(axis="x", tight=True, enable=True)

########################################################################
# Plot the sum of the absolute difference in the SFH (in M_sun) expressed as a % of the total stellar mass 
########################################################################
delta_M = np.array([np.nansum(np.abs(pp_list[ii].sfh_mw_1D - pp_list[ii - 1].sfh_mw_1D)) for ii in range(1, len(pp_list))])
M_tot_vals = np.array([np.nansum(pp.sfh_mw_1D) for pp in pp_list])

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(regul_vals[1:], delta_M / M_tot_vals[1:], marker="D")
ax.grid()
ax.set_xlabel("regul")
ax.set_ylabel(r"$|\Delta M|/M_{\rm tot} \,\rm (M_\odot)$")
ax.set_xscale("log")
ax.set_yscale("log")

########################################################################
# How much does the mass/light-weighted age change as a function of regul?
########################################################################
age_thresh = 1e9
log_mw_age_list = [compute_mw_age(pp.sfh_mw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_list]
log_lw_age_list = [compute_lw_age(pp.sfh_lw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_list]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(regul_vals, log_mw_age_list, marker="D", label=f"Mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.plot(regul_vals, log_lw_age_list, marker="D", label=f"Light-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.grid()
ax.set_xlabel("regul")
ax.set_ylabel("Mean age")
ax.set_xscale("log")
ax.legend()



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in log10
  if __name__ == '__main__':


### Remarks 
---
The MW/LW mean age and the SFH actually continue to change beyond our previously assumed cutoff-value of 5e4. **We therefore need to increase the maximum regularsiation parameter.** 

## How sensitive is the optimal `regul` parameter to random noise at a S/N of 100? 
---
Remarks: the $|\Delta \chi^2 - \Delta \chi^2_{\rm ideal}|$ do change with each random realisation, but not by a significant amount.

In [30]:
########################################################################
# How much does the optimal value of regul change between runs?
########################################################################
pp_list_list = []
for ii in tqdm(range(10)):
    # Re-make the spectrum w/ randomly realised noise
    spec_rand, spec_rand_err, lambda_vals_A = create_mock_spectrum(
        sfh_mass_weighted=sfh_mw_input,
        isochrones=isochrones, z=z, SNR=SNR, sigma_star_kms=sigma_star_kms,
        plotit=False)    
    
    # run ppxf on a grid of fixed regul values
    regul_vals = np.linspace(1e4, 10e4, 20)
    nthreads = len(regul_vals)
    niters = len(regul_vals)
    with multiprocessing.Pool(nthreads) as pool:
        pp_list = list(tqdm(pool.imap(ppxf_helper, regul_vals), total=niters))

    pp_list_list.append(pp_list)


########################################################################
# Plot delta-delta-chi2 vs. regul for each random realisation
########################################################################
fig, ax = plt.subplots(figsize=(10, 5))
for ii, pp_list in enumerate(pp_list_list):
    delta_delta_chi2_vals = np.array([pp.delta_delta_chi2 for pp in pp_list])
    regul_vals = np.array([pp.regul for pp in pp_list])
    ax.plot(regul_vals, delta_delta_chi2_vals, label=f"Iteration {ii}")
    
# Decorations
ax.set_xlabel("regul")
ax.set_ylabel(r"$|\Delta \chi^2 - \Delta \chi^2_{\rm ideal}|$")
ax.legend()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## How small does $\Delta \chi^2 - \Delta \chi^2_{\rm ideal}$ need to be to achieve a satisfactory level of convergence?
---
2 things to test here:
* how much does the optimal value of `regul` change between identical runs? Run ppxf w/ automatic regularisation & compare the SFHs and other derived parameters between runs. This will tell us whether or not we need to re-run ppxf multiple times (like an MC simulation) to get reliable results. 
* how much do the results change for $\Delta \chi^2 - \Delta \chi^2_{\rm ideal} < 1$ vs. $\Delta \chi^2 - \Delta \chi^2_{\rm ideal} = 10$? If it's not a big difference then we can revise `delta_regul_min`.

#### First: does the optimal value for regul change significantly on identical runs?
---
Remarks: not in a significant way. We don't need to worry about this.

In [25]:
########################################################################
# How much does the optimal value of regul change between runs?
########################################################################
pp_opt_list = []
for ii in range(10):
    pp_opt = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
                      z=z, ngascomponents=1, mdegree=-1, reddening=None,
                      regularisation_method="auto", regul_start=1.5e4, regul_max=10e4,
                      fit_agn_cont=False, fit_gas=False, tie_balmer=False,
                      isochrones="Padova")
    pp_opt_list.append(pp_opt)


########################################################################
# Print the regul value from each run
########################################################################
for ii, pp in enumerate(pp_opt_list):
    print(f"Iteration {ii}: Delta-delta-chi2 = {pp.delta_delta_chi2:.4f}, regul_opt = {pp.regul:.1f}")

########################################################################
# How much does the SFH change?
########################################################################
fig = plt.figure(figsize=(10, 3.5))
ax = fig.add_axes([0.1, 0.25, 0.8, 0.65])

# Plot the SFH
cmap_regul = matplotlib.cm.get_cmap("Spectral", len(pp_opt_list))
for ii, pp in enumerate(pp_opt_list):
    ax.step(pp.ages, np.log10(pp.sfh_mw_1D), where="mid", color=cmap_regul(ii), label=f"regul = {pp.regul}")   

# Decorations
ax.set_ylabel(r"Log stellar mass ($\rm \log_{10}\, M_\odot$)")
ax.set_xlabel("Age (yr)")
ax.set_xscale("log")
ax.grid()
ax.legend(fontsize="small")
ax.autoscale(axis="x", tight=True, enable=True)

########################################################################
# How much does the LW/MW age change?
########################################################################
age_thresh = 1e9
log_mw_age_list = [compute_mw_age(pp.sfh_mw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_opt_list]
log_lw_age_list = [compute_lw_age(pp.sfh_lw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_opt_list]

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(range(len(pp_opt_list)), log_mw_age_list, marker="D", label=f"Mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.plot(range(len(pp_opt_list)), log_lw_age_list, marker="D", label=f"Light-weighted age (< {age_thresh / 1e6:.1f} Myr)")
ax.grid()
ax.set_xlabel("Iteration")
ax.set_ylabel("Mean age")
ax.set_xscale("log")
ax.legend()

----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 3.12 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3866...
Iteration 1: Regularisation parameter range: 15000.0-25000.0 (n = 21)
Iteration 1: Running ppxf on 20 threads...


Process ForkPoolWorker-362:
Process ForkPoolWorker-360:
Process ForkPoolWorker-363:
Process ForkPoolWorker-361:
Exception ignored in: <function _releaseLock at 0x7fa8cc2f6d40>
Traceback (most recent call last):
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/logging/__init__.py", line 221, in _releaseLock
    def _releaseLock():
KeyboardInterrupt
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstr

  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Process ForkPoolWorker-368:
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
Process ForkPoolWorker-371:
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/pkg/linux/anaconda-20191122/anaconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
Process ForkPoolWor

KeyboardInterrupt: 

### How much do the SFH, mean age etc. change in a small window around the optimal regul value?
---

In [87]:
################################################################################################
# How much do the SFH, mean age etc. change in a small window around the optimal regul value?
################################################################################################
# Run ppxf on "auto" once to obtain a reasonable estimate for the optimal regul value
pp_opt = run_ppxf(spec=spec, spec_err=spec_err, lambda_vals_A=lambda_vals_A,
              z=z, ngascomponents=1, mdegree=-1, reddening=None,
              regularisation_method="auto",
              fit_agn_cont=False,
              isochrones="Padova",
              fit_gas=False, tie_balmer=True,
              plotit=False, savefigs=False, interactive_mode=False)
regul_opt = pp_opt.regul

----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 2.88 s
----------------------------------------------------
Iteration 1: Scaling noise by 1.3359...
Iteration 1: Regularisation parameter range: 0.0-10000.0 (n = 21)
Iteration 1: Running ppxf on 20 threads...
Iteration 1: Elapsed time in PPXF (multithreaded): 32.12 s
Iteration 1: optimal regul = 1500.00; Δm = 1.83479e+11; Δregul = 500.00 (Δregul_min = 5.00); Δχ (goal) - Δχ = 1.687
----------------------------------------------------
Iteration 2: Re-running ppxf on 20 threads (iteration 2)...
Iteration 2: Regularisation parameter range: 500.0-2500.0 (n = 21)
Iteration 2: Elapsed time in PPXF (multithreaded): 33.88 s
Iteration 2: optimal regul = 1400.00; Δm = 2.78825e+09; Δregul = 100.00 (Δregul_min = 5.00); Δχ (goal) - Δχ = 0.775
----------------------------------------------------
STOPPING: Convergence criterion reached; Δχ (goal) - Δχ = 0.774674648204595; using 1400.00 to produce th

In [88]:
# Re-run ppxf on a grid of regul values centred on the optimal value
regul_vals = np.linspace(0, regul_opt * 2, 20)
nthreads = len(regul_vals)
niters = len(regul_vals)
with multiprocessing.Pool(nthreads) as pool:
    pp_list = list(tqdm(pool.imap(ppxf_helper, regul_vals), total=niters))

----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 10.97 s
Iteration 1: Scaling noise by 1.3554...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 11.30 s
Iteration 1: Scaling noise by 1.3554...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 11.26 s
Iteration 1: Scaling noise by 1.3554...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 11.25 s
Iteration 1: Scaling noise by 1.3554...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 11.42 s
Iteration 1: Scaling noise by 1.3554...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (single thread): 11.61 s
Iteration 1: Scaling noise by 1.3554...
----------------------------------------------------
Iteration 0: Elapsed time in PPXF (

----------------------------------------------------
Best Fit:       Vel     sigma
Desired Delta Chi^2: 95.08
Current Delta Chi^2: 103.7
Delta-Delta Chi^2: 8.615
 comp. 0:        13       103
----------------------------------------------------
chi2/DOF: 1.025
method = capfit ; Jac calls: 3 ; Func calls: 11 ; Status: 4
Nonzero Templates:  170  /  222
----------------------------------------------------
Desired Delta Chi^2: 95.08
Best Fit:       Vel     sigma
Current Delta Chi^2: 113.9
 comp. 0:        13       103
Delta-Delta Chi^2: 18.85
----------------------------------------------------
chi2/DOF: 1.018
Elapsed time in PPXF: 31.71 s
method = capfit ; Jac calls: 3 ; Func calls: 11 ; Status: 4
Nonzero Templates:  155  /  222
----------------------------------------------------
Desired Delta Chi^2: 95.08
Current Delta Chi^2: 82.79
Delta-Delta Chi^2: 12.29
----------------------------------------------------
Elapsed time in PPXF: 31.04 s
Best Fit:       Vel     sigma
 comp. 0:        13

In [89]:
########################################################################
# Plot the delta-delta-chi2 as a function of regul
########################################################################
delta_delta_chi2_vals = np.array([pp.delta_delta_chi2 for pp in pp_list])
regul_vals = np.array([pp.regul for pp in pp_list])

# Find the optimal index 
opt_idx = np.nanargmin(delta_delta_chi2_vals)

fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(regul_vals, delta_delta_chi2_vals, marker="D")
ax.grid()
ax.set_xlabel("regul")
ax.set_ylabel(r"$|\Delta \chi^2 - \Delta \chi^2_{\rm ideal}|$")
ax.set_xscale("log")

########################################################################
# How much does the SFH change?
########################################################################
fig = plt.figure(figsize=(15, 3.5))
ax = fig.add_axes([0.1, 0.25, 0.6, 0.65])

# Plot the SFH
cmap_regul = matplotlib.cm.get_cmap("Spectral", len(pp_list))
for ii, pp in enumerate(pp_list):
    ax.step(pp.ages, np.log10(pp.sfh_mw_1D), where="mid", color=cmap_regul(ii), label=f"delta_delta_chi2 = {pp.delta_delta_chi2:.3f}", lw=0.5)   
ax.step(pp.ages, np.log10(np.nansum(sfh_mw_input, axis=0)), where="mid", color="k", label=f"Input", zorder=999)   
# Decorations
ax.set_ylabel(r"Log stellar mass ($\rm \log_{10}\, M_\odot$)")
ax.set_xlabel("Age (yr)")
ax.set_xscale("log")
ax.grid()
ax.legend(fontsize="x-small", bbox_to_anchor=[1.05, 0.5], loc="center left")
ax.autoscale(axis="x", tight=True, enable=True)

########################################################################
# How much does the LW/MW age change?
########################################################################
for age_thresh in [1e7, 1e8, 1e9, 1e19]:
    # Compute ages
    log_mw_age_input = compute_mw_age(sfh_mw_input, isochrones, age_thresh_upper=age_thresh)[0]
    log_lw_age_input = compute_lw_age(sfh_lw_input, isochrones, age_thresh_upper=age_thresh)[0]
    log_mw_age_list = [compute_mw_age(pp.sfh_mw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_list]
    log_lw_age_list = [compute_lw_age(pp.sfh_lw_1D, isochrones, age_thresh_upper=age_thresh)[0] for pp in pp_list]

    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(regul_vals, log_mw_age_list, marker="D", label=f"Mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
    ax.plot(regul_vals, log_lw_age_list, marker="D", label=f"Light-weighted age (< {age_thresh / 1e6:.1f} Myr)")
    ax.axhline(log_mw_age_input, ls="-", color="k", label="Input mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
    ax.axhline(log_lw_age_input, ls="--", color="k", label="Input mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
    ax.axvline(regul_vals[opt_idx], color="k", label="Optimal regul value")
    ax.grid()
    ax.set_xlabel("regul")
    ax.set_ylabel("Mean age")
    ax.set_xscale("log")
    ax.legend()

    ########################################################################
    # How much does the LW/MW age change as we approach the optimal regul value?
    ########################################################################
    fig, ax = plt.subplots(figsize=(10, 5))
    ax.plot(delta_delta_chi2_vals, log_mw_age_list - log_mw_age_input, marker="D", label=f"Error in mass-weighted age (< {age_thresh / 1e6:.1f} Myr)")
    ax.plot(delta_delta_chi2_vals, log_lw_age_list - log_lw_age_input, marker="D", label=f"Error in light-weighted age (< {age_thresh / 1e6:.1f} Myr)")
    ax.axhline(0, color="black")
    ax.axhspan(ymin=-0.1, ymax=0.1, color="black", alpha=0.2)
    ax.grid()
    ax.set_xlabel("delta-delta-chi2")
    ax.set_ylabel("Mean age")
    ax.legend()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
plt.close("all")

### Remarks
---
* For this particular SFH (gal 1) with no confounding factors, i.e. no emssion lines, no AGN continuum, no extinction etc., the absolute deviation in the mean age (derived for a range of upper age cutoffs) is as large as 0.05 dex for $|\Delta\chi^2 - \Delta\chi^2_{\rm ideal}| \lesssim 5$, perhaps indicating that we should keep our existing convergence criterion.
* Need to re-run these tests for 
    * different SFHs 
    * 